# Database connection test

# Preparation

In [2]:
# Imports
import sqlite3
import pandas as pd

## Main connection

In [3]:
# Connect to main database
db_path = '../TsakonianDB.sqlite3'
conn = sqlite3.connect(db_path)

## Auxiliary functions

In [4]:
def query(query: str,
          cursor = conn.cursor()):

      cursor.execute(query)
      fetched = cursor.fetchall()

      if len(fetched) > 0:
            temp_df = pd.DataFrame(fetched, columns=[i[0] for i in cursor.description])
            return temp_df
      else:
            print('Query executed successfully. No results to show.')

# Add words from main dictionary to the database

## Load dictionary

In [5]:
# Load main dictionary
main_df_path = '../../data/tables/main.xlsx'
main_df = pd.read_excel(main_df_path)
main_df

,tsakonian,greek,paradigm,source_id
0,άγο,άλογο,Α0,1
1,άγουστε,αύγουστος,NaN,1
2,άζ̌α,"αραία, όχι συχνά",NaN,1
3,άθρωπο,άνθροπος,Α1,1
4,άι,λάδι,Υ0,1
...,...,...,...,...
621,όρπα,εκεί,Ζ,1
622,ότσι,ότι,Ζ,1
623,ύο,νερό,Υ0,1
624,μάθημα,μάθημα,Υ,1


## Manage duplicates

In [6]:
# Remove duplicates with the exact same information
print(f'Entries before removing duplicates: {len(main_df)}')
main_df = main_df.drop_duplicates()
print(f'Entries after removing duplicates: {len(main_df)}')

Entries before removing duplicates: 626
Entries after removing duplicates: 626


In [7]:
# Show duplicates
main_df[main_df.duplicated(subset = 'tsakonian', keep = False)].sort_values(by = 'tsakonian')

,tsakonian,greek,paradigm,source_id
140,βου,κλαίω,Ρ,1
141,βου,βόδι,Α2,1
149,βραχάνι,μακρύ φουστάνι,NaN,1
150,βραχάνι,φουστάνι,Υ3,1
274,κηρούνι,πηρούνι,Υ3,1
275,κηρούνι,πηρούνι,NaN,1
365,μπρούσα,τσέπι,Θ1,1
366,μπρούσα,τσέπη,Θ,1


In [8]:
# Keep the Tsakonian-Greek duplicate with the lowest ID
print(f'Entries before removing duplicates: {len(main_df)}')
main_df = (main_df
           .sort_index()
           .drop_duplicates(subset = ['tsakonian', 'greek'], keep = 'first')
           .sort_values(by = 'tsakonian')
           .reset_index(drop = True)
           )
print(f'Entries after removing duplicates: {len(main_df)}')
main_df

Entries before removing duplicates: 626
Entries after removing duplicates: 626


,tsakonian,greek,paradigm,source_id
0,άγο,άλογο,Α0,1
1,άγουστε,αύγουστος,NaN,1
2,άζ̌α,"αραία, όχι συχνά",NaN,1
3,άθρωπο,άνθροπος,Α1,1
4,άι,λάδι,Υ0,1
...,...,...,...,...
621,όντα,δόντι,Υ,1
622,όρεγι,εδώ ακριβώς,Ζ,1
623,όρπα,εκεί,Ζ,1
624,ότσι,ότι,Ζ,1


## Save to main Excel

In [9]:
# Save main_df back to the Excel file
main_df.to_excel(main_df_path, index = False)
print('Changes consolidated.')

Changes consolidated.


## Merge paradigms table

In [10]:
# Read paradigms.xlsx
paradigms_df_path = '../../data/tables/paradigms.xlsx'
paradigms_df = pd.read_excel(paradigms_df_path)
paradigms_df.head()

,paradigm,notes
0,Α0,"ο, pl. irregular"
1,Α1,"ο, pl. -οι"
2,Α2,"ο, pl. -ε"
3,Α3,"ο, pl. -ου"
4,Α4,"ο, pl. -ουνε"


In [12]:
# Merge paradigms_df with main_df
if 'notes' not in main_df.columns:
      main_df = main_df.merge(paradigms_df, how = 'left', on = 'paradigm')

main_df.head()

,tsakonian,greek,paradigm,source_id,notes
0,άγο,άλογο,Α0,1,"ο, pl. irregular"
1,άγουστε,αύγουστος,NaN,1,NaN
2,άζ̌α,"αραία, όχι συχνά",NaN,1,NaN
3,άθρωπο,άνθροπος,Α1,1,"ο, pl. -οι"
4,άι,λάδι,Υ0,1,"το, pl. irregular"


In [13]:
# Copy the full dictionary into the database
main_df.to_sql('dictionary_entry', 
               conn, 
               if_exists='replace', 
               index=False,
               dtype = {'tsakonian': 'varchar(50)',
                        'greek': 'varchar(200)',
                        'paradigm': 'varchar(5)',
                        'source_id': 'bigint',
                        'notes': 'varchar(30)'
               })

# Delete the temporary table if it exists
try:
    query("DROP TABLE sqlitestudio_temp_table;")
except:
    pass

# Add primary keys and foreign keys by recreating the table
# Obtained from SQLiteStudio
recreating_query = """PRAGMA foreign_keys = 0;

CREATE TABLE sqlitestudio_temp_table AS SELECT *
                                          FROM dictionary_entry;

DROP TABLE dictionary_entry;

CREATE TABLE dictionary_entry (
    id        INTEGER       PRIMARY KEY AUTOINCREMENT,
    tsakonian VARCHAR (50)  NOT NULL,
    greek     VARCHAR (200),
    paradigm  VARCHAR (5),
    source_id    INTEGER       REFERENCES dictionary_source (id),
    notes     VARCHAR (30)

);

INSERT INTO dictionary_entry (
                                 tsakonian,
                                 greek,
                                 paradigm,
                                 source_id,
                                 notes
                             )
                             SELECT tsakonian,
                                    greek,
                                    paradigm,
                                    source_id,
                                    notes
                               FROM sqlitestudio_temp_table;

DROP TABLE sqlitestudio_temp_table;

PRAGMA foreign_keys = 1;"""

# Execute queries in a loop
# Only one query can be executed at a time
for q in recreating_query.split(';'):
      query(q)

# Save changes
conn.commit()

Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
